In [ ]:
import json
import os
import pickle

import cv2
import numpy as np
from skimage import (
    color,
    exposure,
    feature,
    filters,
    io,
    measure,
    morphology,
    transform,
)
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.svm import SVC

from constants import LABELS_FILENAME, RANDOM_STATE
from extractor import get_features_from_path
from utils import NumpyEncoder, load_data, show_cv2_image_bgr, show_cv2_image_gray

In [ ]:
path = "data/men/2/2_men (5).JPG"

In [ ]:
img = cv2.imread(path)

In [ ]:
def gray_world(img):
    b, g, r = cv2.split(img)

    b_avg = np.average(b)
    g_avg = np.average(g)
    r_avg = np.average(r)
    avg = (b_avg + g_avg + r_avg) / 3

    b = np.clip(b * (avg / b_avg), 0, 255).astype(np.uint8)
    g = np.clip(g * (avg / g_avg), 0, 255).astype(np.uint8)
    r = np.clip(r * (avg / r_avg), 0, 255).astype(np.uint8)
    return cv2.merge([b, g, r])

In [ ]:
lower = np.array([0, 20, 70])
upper = np.array([20, 255, 255])
kernel = np.ones((5, 5), np.uint8)
# kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8, 8))

In [ ]:
img = gray_world(img)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv, lower, upper)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
hand = cv2.bitwise_and(img, img, mask=mask)

In [ ]:
show_cv2_image_bgr(img)
show_cv2_image_gray(mask)
show_cv2_image_bgr(hand)

In [ ]:
with open(LABELS_FILENAME, "r") as f:
    json_data = json.load(f)

count = 1821
i = 0
for entry in json_data:
    path: str = entry["path"]

    img = cv2.imread(path)
    img = gray_world(img)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    hand = cv2.bitwise_and(img, img, mask=mask)

    show_cv2_image_bgr(hand)